<a href="https://colab.research.google.com/github/HugoLeda/ML-Olympiad-Toxic-Language-PTBR-Detection/blob/main/Training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=0160f86567523d72a909b2a59ea4a70ed1ce9351fa7237069c8efb8aac47b001
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
!wget https://raw.githubusercontent.com/HugoLeda/ML-Olympiad-Toxic-Language-PTBR-Detection/main/assets/train.csv

--2024-05-30 18:18:27--  https://raw.githubusercontent.com/HugoLeda/ML-Olympiad-Toxic-Language-PTBR-Detection/main/assets/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1573871 (1.5M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   1.50M  --.-KB/s    in 0.01s   

2024-05-30 18:18:28 (104 MB/s) - ‘train.csv’ saved [1573871/1573871]



In [ ]:
df = pd.read_csv('train.csv')
df.shape

(16800, 2)

In [ ]:
df.columns

Index(['text', 'label'], dtype='object')

In [ ]:
df.head()

,text,label
0,"rt @user olha quem chegouuuuu, nossos queridin...",0
1,veio umas teorias muito loucas na minha cabeça...,1
2,@user @user 😂😂😂😂mais nao tinha falado ontem qu...,0
3,rt @user quer ser filha da puta logo comigo qu...,1
4,vai besta 😂😂😂😂 casquei com a ultima foto,1


In [ ]:
#clean tweets
def remove_repeated_chars(text):
 return re.sub(r"(.)\1{2,}", r"\1", text)

def cleanText(text):
  text = text.encode('ascii', 'ignore').decode('ascii') #remove emojis
  text = re.sub(r'@\w+', '', text) # remove users mentions
  text = re.sub(r'htttps?//\S+', '', text) #remove links
  text = re.sub(r'\s+', ' ', text) #remove extra spaces
  text = re.sub(r'\b(rt|user|https)\b', '', text, flags=re.IGNORECASE) #remove some words
  text = remove_repeated_chars(text)

  return text.strip()

In [ ]:
#remove stop words
stopWords = set(stopwords.words('portuguese'))
def removeStopWords(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)

  words = text.split()

  filteredWords = [word for word in words if word.lower() not in stopWords]

  cleanedText = ' '.join(filteredWords)

  return cleanedText

In [ ]:
df['text'] = df['text'].apply(cleanText)

df.head(10)

,text,label
0,"olha quem chegou, nossos queridinhos! vem na m...",0
1,veio umas teorias muito loucas na minha cabea ...,1
2,mais nao tinha falado ontem que no ia ser patr...,0
3,quer ser filha da puta logo comigo que sou 50x...,1
4,vai besta casquei com a ultima foto,1
5,"no sei oq mais chocante, um botafoguense, ou u...",1
6,pois mano que coisa chata da porra,0
7,eu odeio sentir dio de algum pq eu t por trs d...,1
8,se eu ganhar raspo minha cabea ://t.co/ucfwfab...,0
9,vlw mano tmj,0


In [ ]:
df['text'] = df['text'].apply(removeStopWords)

df.head(10)

,text,label
0,olha chegou queridinhos vem direo fem 1015 mas...,0
1,veio umas teorias loucas cabea agora pqp to as...,1
2,nao falado ontem ia patrocinado nada pudesse v...,0
3,quer filha puta logo comigo 50x pior k fica sa...,1
4,vai besta casquei ultima foto,1
5,sei oq chocante botafoguense botafoguense fala...,1
6,pois mano coisa chata porra,0
7,odeio sentir dio algum pq t trs porra tela cel...,1
8,ganhar raspo cabea tcoucfwfabmam sorteiochipart,0
9,vlw mano tmj,0


In [ ]:
def portugueseStemmer(text):
  stemmer = SnowballStemmer('portuguese')

  words = text.split()
  stemmedWords = [stemmer.stem(word) for word in text.split()]
  stemmedText = ' '.join(stemmedWords)

  return stemmedText

In [ ]:
df['text'] = df['text'].apply(portugueseStemmer)

df.head(10)

,text,label
0,olha cheg queridinh vem dir fem 1015 masc 2540...,0
1,vei umas teor louc cab agor pqp to assust,1
2,nao fal ontem ia patrocin nad pud viol moral f...,0
3,quer filh put log comig 50x pior k fic sapatinh,1
4,vai best casqu ultim fot,1
5,sei oq chocant botafoguens botafoguens fal tor...,1
6,pois man cois chat porr,0
7,odei sent dio algum pq t trs porr tel celul po...,1
8,ganh rasp cab tcoucfwfabm sorteiochipart,0
9,vlw man tmj,0


In [ ]:
df.to_csv('cleaned_text.csv', index=False)